# Chapter 3: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

In [1]:
!wget https://nlp100.github.io/data/jawiki-country.json.gz -P ../data

--2020-06-23 12:48:28--  https://nlp100.github.io/data/jawiki-country.json.gz
Resolving nlp100.github.io (nlp100.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5068362 (4.8M) [application/gzip]
Saving to: ‘../data/jawiki-country.json.gz’

jawiki-country.json 100%[===================>]   4.83M  2.49MB/s    in 1.9s    

2020-06-23 12:48:33 (2.49 MB/s) - ‘../data/jawiki-country.json.gz’ saved [5068362/5068362]



In [2]:
!gzip -d ../data/jawiki-country.json.gz

../data/jawiki-country.json already exists -- do you wish to overwrite (y or n)? ^C


## 20. JSONデータの読み込み

Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [4]:
import pandas as pd

df = pd.read_json("../data/jawiki-country.json", lines=True)

In [8]:
import json

with open("../data/jawiki-country.json", 'r') as f:
    for line in f:
        data = json.loads(line)
        if data['title'] == 'イギリス':
            text = data['text']
            break

## 21. カテゴリ名を含む行を抽出

記事中でカテゴリ名を宣言している行を抽出せよ．

In [9]:
split_text = text.split('\n')
categories = []
for line in split_text:
    if '[[Category' in line:
        categories.append(line)

for line in categories:
    print(line)

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出

記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [33]:
import re

for line in categories:
    print(re.sub(r'^.*Category:(.*?)(\|.*)?\]\].*$', '\\1', line))

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


## 23. セクション構造

記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [82]:
sections = []
for line in split_text:
    if re.search(r'^=', line):
        title = line.replace('=', '')
        index = (len(line) - len(title)) / 2 - 1
        sections.append((title.strip(), int(index)))
        
for section in sections:
    print(f"{section[0]} {section[1]}")

国名 1
歴史 1
地理 1
主要都市 2
気候 2
政治 1
元首 2
法 2
内政 2
地方行政区分 2
外交・軍事 2
経済 1
鉱業 2
農業 2
貿易 2
不動産 2
エネルギー政策 2
通貨 2
企業 2
通信 3
交通 1
道路 2
鉄道 2
海運 2
航空 2
科学技術 1
国民 1
言語 2
宗教 2
婚姻 2
移住 2
教育 2
医療 2
文化 1
食文化 2
文学 2
哲学 2
音楽 2
ポピュラー音楽 3
映画 2
コメディ 2
国花 2
世界遺産 2
祝祭日 2
スポーツ 2
サッカー 3
クリケット 3
競馬 3
モータースポーツ 3
野球 3
カーリング 3
自転車競技 3
脚注 1
関連項目 1
外部リンク 1


## 24. ファイル参照の抽出

記事から参照されているメディアファイルをすべて抜き出せ．



In [111]:
for line in split_text:
    if '[[ファイル' in line:
        print(re.sub(r'.*\[\[ファイル:(.*\.\w{3})[\|\]].*', '\\1', line))

Royal Coat of Arms of the United Kingdom.svg
United States Navy Band - God Save the Queen.ogg
Descriptio Prime Tabulae Europae.jpg
Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
London.bankofengland.arp.jpg
Battle of Waterloo 1815.PNG
Uk topo en.jpg
BenNevis2005.jpg
Population density UK 2011 census.png
Birmingham Skyline from Edgbaston Cricket Ground crop.jpg
Glasgow and the Clyde from the air (geograph 4665720).jpg
Palace of Westminster, London - Feb 2007.jpg
Scotland Parliament Holyrood.jpg
Donald Trump and Theresa May (33998675310) (cropped).jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Airbus A380-841 G-XLEB British Airways (10424102995).jpg
UKpop.svg
Anglospeak.svg
Royal Aberdeen Children's Hospital.jpg
CHANDOS3.jpg
The Fabs.JPG
Wembley Stadium, illuminated.jpg


## 25. テンプレートの抽出

記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [43]:
import re

pattern = re.compile(r"\n\{\{基礎情報.+\n\}\}", re.DOTALL)

for match in pattern.finditer(text):
    info = match.group(0)

In [44]:
info_split = info.split('\n|')

In [53]:
pattern = re.compile(r"(.*?)=(.*)")

info_dict = {}
for info in info_split:
    match = re.match(pattern, info)
    if match:
        info_dict[match.group(1).strip()] = match.group(2).strip()

In [54]:
info_dict

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />''神よ女王を護り賜え''<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}",
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Linds

## 強調マークアップの除去

25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [55]:
for key in info_dict:
    info_dict[key] = info_dict[key].replace("'", "")
    
info_dict

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイル:United States Navy Band - God Save the Queen.ogg]]}}',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '[[英語]]',
 '首都': '[[ロンドン]]（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '[[イギリスの君主|女王]]',
 '元首等氏名': '[[エリザベス2世]]',
 '首相等肩書': '[[イギリスの首相|首相]]',
 '首相等氏名': '[[ボリス・ジョンソン]]',
 '他元首等肩書1': '[[貴族院 (イギリス)|貴族院議長]]',
 '他元首等氏名1': '[[:en:Norman Fowler, Baron Fowler|ノーマン・ファウラー]]',
 '他元首等肩書2': '[[庶民院 (イギリス)|庶民院議長]]',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay H

## 27. 内部リンクの除去

26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [107]:
pattern = re.compile(r'\[\[?.*\|*(.*)?\]\]?')

for key, val in info_dict.items():
    res = re.findall(pattern, val)
    if res:
        print(res)
    

['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']


## 28. MediaWikiマークアップの除去

27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

## 29. 国旗画像のURLを取得する

テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](https://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して，ファイル参照をURLに変換すればよい）